# Syn vs Nonsyn and Non-nonsense vs Nonsense mutation rates

## Based on positions, instead of codons!

This makes it easier to use these for FDR estimation.

In [3]:
%run "Header.ipynb"
%run "GeneUtils.ipynb"

In [61]:
import time
import json
import pickle
import skbio
import pileup
from collections import defaultdict
from parse_sco import parse_sco

In [5]:
seq2pos2pileup = pileup.load()

## Percentages (values of $p$) for naive mutation calling

### Values we use for plotting
Currently the same as in the codon mutation notebook. I guess these should be a shared variable, ideally? But that might be too much work for its own good.

In [6]:
percentages = [2, 1, 0.5, 0.25, 0.15]

### Full values, used for plotting a FDR curve elsewhere

Copied from the target/decoy notebook. may be nice to generalize to shared code too i guess

In [7]:
# Percentages go from 4.99%, 4.98%, ..., 0.16%, 0.15%
full_percentages = [p / 100 for p in range(15, 500, 1)][::-1]
print(f"First two percentages: {full_percentages[:2]}")
print(f"Last two percentages: {full_percentages[-2:]}")
print(f"Number of percentages: {len(full_percentages):,}")

First two percentages: [4.99, 4.98]
Last two percentages: [0.16, 0.15]
Number of percentages: 485


In [8]:
# Sanity check -- we can just compute this data for all of the stuff in full_percentages
# and then for the stuff we wanna plot in this notebook just focus on the values in percentages
for p in percentages:
    assert p in full_percentages, f"{p}% not in full_percentages"

## Compute possible (non)synonymous and (non)nonsense mutations, in general

Not specific to a given MAG -- just in the context of all possible codons, using the standard genetic code.

In [9]:
codons = []
dna = "ACGT"
for x in dna:
    for y in dna:
        for z in dna:
            codons.append(x+y+z)
            
# Maps the 64 codons --> [1, 2, 3] --> an integer 0, 1, 2, or 3, indicating how many of the 3 mutations
# from this CP in this codon into another nucleotide are synonymous (si) or nonsynonymous (ni)
codon2cp2si = defaultdict(dict)
codon2cp2ni = defaultdict(dict)

# Like above, but for non-nonsense (nnsi) or nonsense (nsi) mutations and only for the 61 sense codons
codon2cp2nnsi = defaultdict(dict)
codon2cp2nsi = defaultdict(dict)

stop_codons = []

for c in codons:
    is_not_stop = not (str(skbio.DNA(c).translate()) == "*")
    if not is_not_stop:
        stop_codons.append(c)
    for pos in [0, 1, 2]:
        si = 0
        ni = 0
        nnsi = 0
        nsi = 0
        for alt_nt in set(dna) - set(c[pos]):
            alt_codon = c[:pos] + alt_nt + c[pos + 1:]
            aa1 = str(skbio.DNA(c).translate())
            aa2 = str(skbio.DNA(alt_codon).translate())
            if aa1 == aa2:
                si += 1
                if is_not_stop:
                    nnsi += 1
            else:
                ni += 1
                if is_not_stop:
                    if aa2 == "*":
                        nsi += 1
                    else:
                        nnsi += 1
                    
        assert si + ni == 3
        codon2cp2si[c][pos + 1] = si
        codon2cp2ni[c][pos + 1] = ni
        
        if is_not_stop:
            assert nsi + nnsi == 3
            codon2cp2nnsi[c][pos + 1] = nnsi
            codon2cp2nsi[c][pos + 1] = nsi
            
assert len(codon2cp2si) == len(codon2cp2ni) == 64
assert len(codon2cp2nnsi) == len(codon2cp2nsi) == 61
assert len(stop_codons) == 3

## Use these numbers to compute $\sum_i S_i$ (and $N_i$, $NNS_i$, $NS_i$) for all MAGs

In [62]:
# seq --> list of 1-indexed positions where a mutation here can be nonsyn or nonsense
seq2poss_nonsyn_positions = {}
seq2poss_nonsense_positions = {}

# Total poss numbers of single-nucleotide (non)synonymous and (non)nonsense mutations throughout the genomes
seq2poss_si = defaultdict(int)
seq2poss_ni = defaultdict(int)
seq2poss_nnsi = defaultdict(int)
seq2poss_nsi = defaultdict(int)

# Defined the same as in the earlier codon-based S/N notebook.
p2seq2obs_si = {p: defaultdict(int) for p in full_percentages}
p2seq2obs_ni = {p: defaultdict(int) for p in full_percentages}
p2seq2obs_nnsi = {p: defaultdict(int) for p in full_percentages}
p2seq2obs_nsi = {p: defaultdict(int) for p in full_percentages}

# For debugging... I don't recommend using this otherwise due to how much gets printed out!
verbose = False

t0 = time.time()
for seq in SEQS:
    print(f"On {seq}. Runtime: {time.time() - t0:,.2f} sec.")
    df = parse_sco(f"../seqs/genes/{seq}.sco")
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    # Figure out quickly which positions are in multiple genes, so we can ignore them
    pos_to_genes = get_parent_gene_info_of_many_positions(df)
    
    poss_nonsyn_pos = []
    poss_nonsense_pos = []
    
    for gene in df.itertuples():
        t0g = time.time()
        print(
            f"{seq2name[seq]}: On gene {gene.Index:,} / {len(df.index):,}. "
            f"So far {len(poss_nonsyn_pos):,} poss nonsyn and {len(poss_nonsense_pos):,} "
            f"poss nonsense positions. Runtime: {time.time() - t0:,.2f} sec."
        )
        # The order of positions we go through is dependent on gene orientation:
        # left --> right if this is a "+" strand gene,
        # right --> left if this is a "-" strand gene.
        pos_interval = get_pos_interval_from_gene(gene)
        
        # We'll update these values as we walk through the gene.
        # curr_codon_cp1_pos indicates CP1 (the left end for + genes, the right end for - genes)
        # of the current gene. It's 1-indexed, along with the gene coordinates.
        curr_codon_cp1_pos = pos_interval[0]
        # Always either 1, 2, or 3
        cp = 1
        
        for pos in pos_interval:
            if verbose: print(f"On position {pos:,} (CP {cp}) ")
            if cp == 1:
                # It's easier to sync this at the top rather than bottom of the loop because the positions
                # in pos_interval could be increasing or decreasing depending on the gene's strand
                curr_codon_cp1_pos = pos
            
            # We don't attempt mutation calling at multi-gene positions, or at
            # unreasonable (ref != consensus) positions.
            
            # ... And now we actually naively try to call a mutation at this position (trying lots of
            # values of p at once so we can create an FDR curve, barplots, etc).
            pos_pileup = seq2pos2pileup[seq][pos]
            
            if len(pos_to_genes[pos]) == 1 and pileup.is_reasonable(pos_pileup):
                
                if verbose: print("\tReasonable and in just 1 gene ", end="")
                    
                if gene.Strand == "+":
                    # The skbio sequence is 0-indexed, so we gotta subtract by 1
                    parent_codon_fasta = fasta[curr_codon_cp1_pos - 1: curr_codon_cp1_pos + 2]
                    if verbose:
                        print(f"(+); parent codon is {parent_codon_fasta}")
                    
                elif gene.Strand == "-":
                    parent_codon_fwd_fasta = fasta[curr_codon_cp1_pos - 3: curr_codon_cp1_pos]
                    parent_codon_fwd_str = str(parent_codon_fwd_fasta)
                    parent_codon_fasta = parent_codon_fwd_fasta.reverse_complement()
                    if verbose:
                        print(
                            f"(-); parent codon (not RC'd) is {parent_codon_fwd_fasta}; "
                            f"RC'd, it's {parent_codon_fasta} ({parent_codon_fasta.translate()})"
                        )
                
                else:
                    raise ValueError(f"Invalid strand: {gene.Strand}")
                
                parent_codon_str = str(parent_codon_fasta)
                
                # Update possible Si|Ni and NNSi|NSi values based on this position and its parent codon.
                seq2poss_si[seq] += codon2cp2si[parent_codon_str][cp]
                seq2poss_ni[seq] += codon2cp2ni[parent_codon_str][cp]
                if codon2cp2ni[parent_codon_str][cp] > 0:
                    poss_nonsyn_pos.append(pos)
                    
                if verbose:
                    print(
                        f"\tSi = {codon2cp2si[parent_codon_str][cp]}, "
                        f"Ni = {codon2cp2ni[parent_codon_str][cp]}"
                    )
                    
                in_sense_codon = (parent_codon_str not in stop_codons)
                if in_sense_codon:
                    seq2poss_nnsi[seq] += codon2cp2nnsi[parent_codon_str][cp]
                    seq2poss_nsi[seq] += codon2cp2nsi[parent_codon_str][cp]
                    
                    if codon2cp2nsi[parent_codon_str][cp] > 0:
                        poss_nonsense_pos.append(pos)
                        
                    if verbose:
                        print(
                            f"\tPosition is in a sense codon, and "
                            f"NNSi = {codon2cp2nnsi[parent_codon_str][cp]}, "
                            f"NSi = {codon2cp2nsi[parent_codon_str][cp]}"
                        )
                
                # Since we currently define mutations as a binary thing (either a position is mutated
                # or it isn't), we can precompute the step of figuring out the mutated codon resulting from
                # a position being called a mutation -- and we can thus figure out in advance if this position
                # will be a (non)synonymous or (non)nonsense mutation, before actually calling it a mutation
                # (...should save some time).

                if pileup.any_mismatches(pos_pileup):
                    alt_nt = pileup.get_alt_nt_if_reasonable(pos_pileup)

                    if verbose: print(f"\tPosition has mismatches; max freq alt nt is {alt_nt}")

                    # if alt_nt is None, it means this position was unreasonable -- but we've already
                    # verified above that this position is reasonable!
                    assert alt_nt is not None

                    # Construct a "mutated" version of this position's parent codon, with just this
                    # alternate nucleotide changed. This doesn't take into account the possibility
                    # of other positions within this codon being mutated, which is a notable limitation
                    # of this approach.
                    if gene.Strand == "+":
                        mutated_parent_codon = parent_codon_fasta.replace([cp - 1], alt_nt)
                    else:
                        # Do two things: 1) convert from a reverse-direction CP to a fwd-direction CP,
                        # and 2) switch to using 0-indexing so we can work with skbio.DNA objects.
                        if cp == 1: idx_to_replace = 2
                        elif cp == 2: idx_to_replace = 1
                        else: idx_to_replace = 0

                        # Apply the mutation, *then* reverse-complement the codon
                        mutated_parent_codon_fwd = parent_codon_fwd_fasta.replace([idx_to_replace], alt_nt)
                        mutated_parent_codon = mutated_parent_codon_fwd.reverse_complement()
                
                    aa1 = str(parent_codon_fasta.translate())
                    aa2 = str(mutated_parent_codon.translate())
                    
                    is_syn = (aa1 == aa2)
                    aa2_is_stop = (aa2 == "*")
                    
                    if verbose:
                        print(f"\tAlt codon is {mutated_parent_codon} ({aa2}); is syn? {is_syn}")
                    
                    # We don't go through this step if pileup.any_mismatches() is False! Because... in that
                    # case, this position can never be a p-mutation. So we can save a lot of time.
                    for p in full_percentages:

                        if pileup.naively_call_mutation(pos_pileup, p, only_call_if_rare=True):
                            if verbose:
                                print(f"\tIs (rare) mut at p = {p}!")
                            if is_syn:
                                p2seq2obs_si[p][seq] += 1
                            else:
                                p2seq2obs_ni[p][seq] += 1

                            if in_sense_codon:
                                if aa2_is_stop:
                                    p2seq2obs_nsi[p][seq] += 1
                                else:
                                    p2seq2obs_nnsi[p][seq] += 1
                        
            cp = next_cp(cp)
    
    seq2poss_nonsyn_positions[seq] = poss_nonsyn_pos
    seq2poss_nonsense_positions[seq] = poss_nonsense_pos
    
print(f"Done. Runtime: {time.time() - t0:,.2f} sec.")

On edge_6104. Runtime: 0.00 sec.
CAMP: On gene 1 / 1,297. So far 0 poss nonsyn and 0 poss nonsense positions. Runtime: 0.48 sec.
CAMP: On gene 2 / 1,297. So far 400 poss nonsyn and 58 poss nonsense positions. Runtime: 0.80 sec.
CAMP: On gene 3 / 1,297. So far 736 poss nonsyn and 113 poss nonsense positions. Runtime: 1.06 sec.
CAMP: On gene 4 / 1,297. So far 1,126 poss nonsyn and 168 poss nonsense positions. Runtime: 1.39 sec.
CAMP: On gene 5 / 1,297. So far 2,283 poss nonsyn and 315 poss nonsense positions. Runtime: 2.29 sec.
CAMP: On gene 6 / 1,297. So far 2,516 poss nonsyn and 349 poss nonsense positions. Runtime: 2.47 sec.
CAMP: On gene 7 / 1,297. So far 3,450 poss nonsyn and 472 poss nonsense positions. Runtime: 3.12 sec.
CAMP: On gene 8 / 1,297. So far 3,936 poss nonsyn and 525 poss nonsense positions. Runtime: 3.48 sec.
CAMP: On gene 9 / 1,297. So far 4,625 poss nonsyn and 618 poss nonsense positions. Runtime: 3.96 sec.
CAMP: On gene 10 / 1,297. So far 5,076 poss nonsyn and 679 p

CAMP: On gene 78 / 1,297. So far 59,465 poss nonsyn and 7,682 poss nonsense positions. Runtime: 43.91 sec.
CAMP: On gene 79 / 1,297. So far 59,789 poss nonsyn and 7,732 poss nonsense positions. Runtime: 44.11 sec.
CAMP: On gene 80 / 1,297. So far 60,168 poss nonsyn and 7,786 poss nonsense positions. Runtime: 44.34 sec.
CAMP: On gene 81 / 1,297. So far 61,062 poss nonsyn and 7,906 poss nonsense positions. Runtime: 44.98 sec.
CAMP: On gene 82 / 1,297. So far 61,515 poss nonsyn and 7,959 poss nonsense positions. Runtime: 45.32 sec.
CAMP: On gene 83 / 1,297. So far 62,415 poss nonsyn and 8,085 poss nonsense positions. Runtime: 45.91 sec.
CAMP: On gene 84 / 1,297. So far 63,213 poss nonsyn and 8,183 poss nonsense positions. Runtime: 46.50 sec.
CAMP: On gene 85 / 1,297. So far 64,151 poss nonsyn and 8,306 poss nonsense positions. Runtime: 47.20 sec.
CAMP: On gene 86 / 1,297. So far 64,802 poss nonsyn and 8,400 poss nonsense positions. Runtime: 47.64 sec.
CAMP: On gene 87 / 1,297. So far 65,2

CAMP: On gene 154 / 1,297. So far 116,984 poss nonsyn and 14,919 poss nonsense positions. Runtime: 78.95 sec.
CAMP: On gene 155 / 1,297. So far 117,463 poss nonsyn and 14,968 poss nonsense positions. Runtime: 79.29 sec.
CAMP: On gene 156 / 1,297. So far 118,460 poss nonsyn and 15,097 poss nonsense positions. Runtime: 79.99 sec.
CAMP: On gene 157 / 1,297. So far 119,391 poss nonsyn and 15,212 poss nonsense positions. Runtime: 80.64 sec.
CAMP: On gene 158 / 1,297. So far 119,568 poss nonsyn and 15,232 poss nonsense positions. Runtime: 80.75 sec.
CAMP: On gene 159 / 1,297. So far 120,999 poss nonsyn and 15,433 poss nonsense positions. Runtime: 81.76 sec.
CAMP: On gene 160 / 1,297. So far 122,026 poss nonsyn and 15,547 poss nonsense positions. Runtime: 82.48 sec.
CAMP: On gene 161 / 1,297. So far 123,028 poss nonsyn and 15,676 poss nonsense positions. Runtime: 83.24 sec.
CAMP: On gene 162 / 1,297. So far 123,291 poss nonsyn and 15,708 poss nonsense positions. Runtime: 83.39 sec.
CAMP: On g

CAMP: On gene 229 / 1,297. So far 180,456 poss nonsyn and 22,935 poss nonsense positions. Runtime: 123.30 sec.
CAMP: On gene 230 / 1,297. So far 181,257 poss nonsyn and 23,061 poss nonsense positions. Runtime: 123.82 sec.
CAMP: On gene 231 / 1,297. So far 182,409 poss nonsyn and 23,201 poss nonsense positions. Runtime: 124.53 sec.
CAMP: On gene 232 / 1,297. So far 184,307 poss nonsyn and 23,426 poss nonsense positions. Runtime: 125.70 sec.
CAMP: On gene 233 / 1,297. So far 184,961 poss nonsyn and 23,518 poss nonsense positions. Runtime: 126.12 sec.
CAMP: On gene 234 / 1,297. So far 186,303 poss nonsyn and 23,661 poss nonsense positions. Runtime: 127.01 sec.
CAMP: On gene 235 / 1,297. So far 187,697 poss nonsyn and 23,829 poss nonsense positions. Runtime: 127.88 sec.
CAMP: On gene 236 / 1,297. So far 188,045 poss nonsyn and 23,873 poss nonsense positions. Runtime: 128.11 sec.
CAMP: On gene 237 / 1,297. So far 190,101 poss nonsyn and 24,100 poss nonsense positions. Runtime: 129.39 sec.
C

CAMP: On gene 303 / 1,297. So far 247,521 poss nonsyn and 31,454 poss nonsense positions. Runtime: 168.43 sec.
CAMP: On gene 304 / 1,297. So far 248,638 poss nonsyn and 31,580 poss nonsense positions. Runtime: 169.16 sec.
CAMP: On gene 305 / 1,297. So far 250,371 poss nonsyn and 31,839 poss nonsense positions. Runtime: 170.25 sec.
CAMP: On gene 306 / 1,297. So far 251,558 poss nonsyn and 31,956 poss nonsense positions. Runtime: 170.99 sec.
CAMP: On gene 307 / 1,297. So far 252,600 poss nonsyn and 32,080 poss nonsense positions. Runtime: 171.67 sec.
CAMP: On gene 308 / 1,297. So far 253,300 poss nonsyn and 32,183 poss nonsense positions. Runtime: 172.11 sec.
CAMP: On gene 309 / 1,297. So far 253,716 poss nonsyn and 32,230 poss nonsense positions. Runtime: 172.38 sec.
CAMP: On gene 310 / 1,297. So far 254,069 poss nonsyn and 32,278 poss nonsense positions. Runtime: 172.63 sec.
CAMP: On gene 311 / 1,297. So far 255,078 poss nonsyn and 32,390 poss nonsense positions. Runtime: 173.36 sec.
C

...

BACT2: On gene 2,282 / 2,567. So far 1,702,931 poss nonsyn and 202,064 poss nonsense positions. Runtime: 2,340.28 sec.
BACT2: On gene 2,283 / 2,567. So far 1,703,063 poss nonsyn and 202,076 poss nonsense positions. Runtime: 2,340.37 sec.
BACT2: On gene 2,284 / 2,567. So far 1,704,118 poss nonsyn and 202,207 poss nonsense positions. Runtime: 2,341.14 sec.
BACT2: On gene 2,285 / 2,567. So far 1,704,351 poss nonsyn and 202,243 poss nonsense positions. Runtime: 2,341.31 sec.
BACT2: On gene 2,286 / 2,567. So far 1,706,459 poss nonsyn and 202,550 poss nonsense positions. Runtime: 2,342.85 sec.
BACT2: On gene 2,287 / 2,567. So far 1,707,979 poss nonsyn and 202,749 poss nonsense positions. Runtime: 2,344.00 sec.
BACT2: On gene 2,288 / 2,567. So far 1,708,222 poss nonsyn and 202,788 poss nonsense positions. Runtime: 2,344.14 sec.
BACT2: On gene 2,289 / 2,567. So far 1,708,931 poss nonsyn and 202,877 poss nonsense positions. Runtime: 2,344.62 sec.
BACT2: On gene 2,290 / 2,567. So far 1,709,474 p

BACT2: On gene 2,351 / 2,567. So far 1,748,912 poss nonsyn and 208,262 poss nonsense positions. Runtime: 2,370.47 sec.
BACT2: On gene 2,352 / 2,567. So far 1,749,255 poss nonsyn and 208,307 poss nonsense positions. Runtime: 2,370.68 sec.
BACT2: On gene 2,353 / 2,567. So far 1,749,969 poss nonsyn and 208,392 poss nonsense positions. Runtime: 2,371.16 sec.
BACT2: On gene 2,354 / 2,567. So far 1,750,351 poss nonsyn and 208,447 poss nonsense positions. Runtime: 2,371.41 sec.
BACT2: On gene 2,355 / 2,567. So far 1,750,861 poss nonsyn and 208,514 poss nonsense positions. Runtime: 2,371.74 sec.
BACT2: On gene 2,356 / 2,567. So far 1,751,262 poss nonsyn and 208,570 poss nonsense positions. Runtime: 2,372.00 sec.
BACT2: On gene 2,357 / 2,567. So far 1,751,878 poss nonsyn and 208,662 poss nonsense positions. Runtime: 2,372.43 sec.
BACT2: On gene 2,358 / 2,567. So far 1,753,107 poss nonsyn and 208,799 poss nonsense positions. Runtime: 2,373.35 sec.
BACT2: On gene 2,359 / 2,567. So far 1,753,357 p

BACT2: On gene 2,420 / 2,567. So far 1,791,721 poss nonsyn and 213,260 poss nonsense positions. Runtime: 2,395.54 sec.
BACT2: On gene 2,421 / 2,567. So far 1,792,133 poss nonsyn and 213,325 poss nonsense positions. Runtime: 2,395.81 sec.
BACT2: On gene 2,422 / 2,567. So far 1,792,461 poss nonsyn and 213,367 poss nonsense positions. Runtime: 2,396.00 sec.
BACT2: On gene 2,423 / 2,567. So far 1,793,017 poss nonsyn and 213,450 poss nonsense positions. Runtime: 2,396.26 sec.
BACT2: On gene 2,424 / 2,567. So far 1,793,733 poss nonsyn and 213,552 poss nonsense positions. Runtime: 2,396.60 sec.
BACT2: On gene 2,425 / 2,567. So far 1,794,452 poss nonsyn and 213,631 poss nonsense positions. Runtime: 2,397.00 sec.
BACT2: On gene 2,426 / 2,567. So far 1,794,895 poss nonsyn and 213,681 poss nonsense positions. Runtime: 2,397.27 sec.
BACT2: On gene 2,427 / 2,567. So far 1,795,340 poss nonsyn and 213,745 poss nonsense positions. Runtime: 2,397.49 sec.
BACT2: On gene 2,428 / 2,567. So far 1,795,788 p

BACT2: On gene 2,489 / 2,567. So far 1,851,370 poss nonsyn and 220,298 poss nonsense positions. Runtime: 2,429.05 sec.
BACT2: On gene 2,490 / 2,567. So far 1,851,596 poss nonsyn and 220,323 poss nonsense positions. Runtime: 2,429.18 sec.
BACT2: On gene 2,491 / 2,567. So far 1,852,948 poss nonsyn and 220,464 poss nonsense positions. Runtime: 2,429.92 sec.
BACT2: On gene 2,492 / 2,567. So far 1,854,168 poss nonsyn and 220,576 poss nonsense positions. Runtime: 2,430.63 sec.
BACT2: On gene 2,493 / 2,567. So far 1,854,911 poss nonsyn and 220,678 poss nonsense positions. Runtime: 2,431.02 sec.
BACT2: On gene 2,494 / 2,567. So far 1,855,848 poss nonsyn and 220,782 poss nonsense positions. Runtime: 2,431.53 sec.
BACT2: On gene 2,495 / 2,567. So far 1,858,738 poss nonsyn and 221,120 poss nonsense positions. Runtime: 2,433.33 sec.
BACT2: On gene 2,496 / 2,567. So far 1,859,043 poss nonsyn and 221,156 poss nonsense positions. Runtime: 2,433.49 sec.
BACT2: On gene 2,497 / 2,567. So far 1,862,013 p

BACT2: On gene 2,558 / 2,567. So far 1,910,308 poss nonsyn and 226,902 poss nonsense positions. Runtime: 2,461.92 sec.
BACT2: On gene 2,559 / 2,567. So far 1,910,632 poss nonsyn and 226,942 poss nonsense positions. Runtime: 2,462.11 sec.
BACT2: On gene 2,560 / 2,567. So far 1,911,290 poss nonsyn and 227,018 poss nonsense positions. Runtime: 2,462.55 sec.
BACT2: On gene 2,561 / 2,567. So far 1,911,957 poss nonsyn and 227,089 poss nonsense positions. Runtime: 2,463.02 sec.
BACT2: On gene 2,562 / 2,567. So far 1,912,713 poss nonsyn and 227,167 poss nonsense positions. Runtime: 2,463.50 sec.
BACT2: On gene 2,563 / 2,567. So far 1,913,784 poss nonsyn and 227,305 poss nonsense positions. Runtime: 2,464.13 sec.
BACT2: On gene 2,564 / 2,567. So far 1,914,311 poss nonsyn and 227,379 poss nonsense positions. Runtime: 2,464.43 sec.
BACT2: On gene 2,565 / 2,567. So far 1,916,331 poss nonsyn and 227,641 poss nonsense positions. Runtime: 2,465.59 sec.
BACT2: On gene 2,566 / 2,567. So far 1,917,073 p

In [68]:
p2seq2obs_si[0.15]

defaultdict(int, {'edge_6104': 767, 'edge_1671': 13674, 'edge_2358': 4336})

In [69]:
p2seq2obs_ni[0.15]

defaultdict(int, {'edge_6104': 4946, 'edge_1671': 21195, 'edge_2358': 12044})

In [70]:
with open("misc-output/pos_p2seq2obs_si.pickle", "wb") as pf:
    pickle.dump(p2seq2obs_si, pf)
    
with open("misc-output/pos_p2seq2obs_ni.pickle", "wb") as pf:
    pickle.dump(p2seq2obs_ni, pf)
    
with open("misc-output/pos_p2seq2obs_nnsi.pickle", "wb") as pf:
    pickle.dump(p2seq2obs_nnsi, pf)
    
with open("misc-output/pos_p2seq2obs_nsi.pickle", "wb") as pf:
    pickle.dump(p2seq2obs_nsi, pf)

with open("misc-output/pos_seq2poss_si.pickle", "wb") as pf:
    pickle.dump(seq2poss_si, pf)
    
with open("misc-output/pos_seq2poss_ni.pickle", "wb") as pf:
    pickle.dump(seq2poss_ni, pf)
    
with open("misc-output/pos_seq2poss_nnsi.pickle", "wb") as pf:
    pickle.dump(seq2poss_nnsi, pf)

with open("misc-output/pos_seq2poss_nsi.pickle", "wb") as pf:
    pickle.dump(seq2poss_nsi, pf)
    
with open("misc-output/seq2poss_nonsyn_positions.pickle", "wb") as pf:
    pickle.dump(seq2poss_nonsyn_positions, pf)

with open("misc-output/seq2poss_nonsense_positions.pickle", "wb") as pf:
    pickle.dump(seq2poss_nonsense_positions, pf)

In [67]:
for seq in SEQS:
    n = len(seq2poss_nonsyn_positions[seq])
    ns = len(seq2poss_nonsense_positions[seq])
    ratio = n / ns
    print(
        f"{seq}: {n:,} possible nonsyn and "
        f"{ns:,} possible nonsense mutations. Ratio of #nonsyn to #nonsense: {ratio:.2f}"
    )

edge_6104: 1,045,732 possible nonsyn and 132,806 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 7.87
edge_1671: 1,684,876 possible nonsyn and 240,522 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 7.01
edge_2358: 1,917,610 possible nonsyn and 227,810 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 8.42
